In [1]:
import os
from pathlib import Path

In [1]:
pwd

'c:\\Users\\bhand\\OneDrive\\Desktop\\Deep Learning\\Nidarshan\\research'

In [2]:
cd ..

c:\Users\bhand\OneDrive\Desktop\Deep Learning\Nidarshan


In [32]:
pwd

'c:\\Users\\bhand\\OneDrive\\Desktop\\Deep Learning\\Nidarshan'

In [33]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [34]:
from Nidarshan.constants import *
from Nidarshan.utils.common import read_yaml, create_directories

In [35]:
from pathlib import Path
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = Path(CONFIG_FILE_PATH),
        params_filepath = Path(PARAMS_FILE_PATH)):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
        
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_url = config.source_url,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )
        
        return data_ingestion_config

In [36]:
import os
import urllib.request as request
import zipfile
from Nidarshan import logger
from Nidarshan.utils.common import get_size

In [37]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_url,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")
            
    
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
            zip_ref.extractall(unzip_path)

In [38]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-12-10 16:11:10,783: INFO: common: yaml file: C:\Users\bhand\OneDrive\Desktop\Deep Learning\Nidarshan\config\config.yaml loaded successfully]
[2023-12-10 16:11:10,785: INFO: common: yaml file: C:\Users\bhand\OneDrive\Desktop\Deep Learning\Nidarshan\params.yaml loaded successfully]
[2023-12-10 16:11:10,786: INFO: common: Create directory at: artifacts]
[2023-12-10 16:11:10,787: INFO: common: Create directory at: artifacts/data_ingestion]
[2023-12-10 16:11:18,251: INFO: 1457973317: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 49213740
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "5233545dbdacd20ab80ba163c69c0c3a0bb45d9efe54f9d34bb13251bec4a6c5"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 623A:45F9:7A14E9:97D8C2:657595C5
Accept-